In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [3]:
# Initialize

openai = OpenAI()
MODEL = "gpt-4o-mini"

In [4]:
system_message = "You are a helpful assistant"

## Reminder of the structure of prompt messages to OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

We will write a function `chat(message, history)` where:
**message** is the prompt to use
**history** is a list of pairs of user message with assistant's reply

```
[
    ["user said this", "assistant replied"],
    ["then user said this", "and assistant replied again],
    ...
]
```
We will convert this history into the prompt style for OpenAI, then call OpenAI. 

In [5]:
def chat(message, history):
    # message: The latest message input from the user.
    # history: A list of tuples containing previous user and assistant messages.

    # Initialize the conversation with a system message that sets the assistant's behavior.
    # The OpenAI API expects messages as a list of dictionaries with "role" and "content" keys.
    messages = [{"role": "system", "content": system_message}]

    # The conversation history is stored as a list of tuples, where each tuple contains a user message and the corresponding assistant reply.
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assitant_message})
    # Add the latest user message to the conversation.
    messages.append({"role": "user", "content": message})

    print("History is: ")
    print(history)
    print("And messages is: ")
    print(messages)

    stream = openai.chat.completions.create(model = MODEL, messages = messages, stream = True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response
    

### **Function Callback:**
* Gradio uses the **fn** parameter to know which function to invoke for processing inputs.
### **User Interaction Flow:**
* User types a message in the chat interface.
* Gradio passes this message to the **chat** function.
* The **chat** function processes the message and returns a response.
* Gradio displays the assistant's response in the chat interface.
### **.launch()** : What It Does 
* **Launches the Interface:** Starts a local web server and opens the chat interface in a new browser tab or inline (if in a Jupyter environment).
* **Makes the App Accessible:** Allows users to interact with your chatbot via a web interface.

In [8]:
# gr.ChatInterface: A class provided by the Gradio library specifically designed for creating chat-based interfaces.
# gr.ChatInterface(fn=chat): Creates a chat interface bound to your chat function, which processes messages.
# .launch(): Starts the Gradio app, making the chat interface accessible to users.
gr.ChatInterface(fn = chat, type = "tuples").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


History is: 
[]
And messages is: 
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What kind of nice features dose 2025 Kia Sportage EX have?'}]


In [6]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [13]:
gr.ChatInterface(fn = chat, type = "tuples").launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


History is: 
[]
And messages is: 
[{'role': 'system', 'content': 'You are a helpful assistant\nIf the customer asks for shoes, you should respond that shoes are not on sale today, but remind the customer to look at hats!'}, {'role': 'user', 'content': 'I want to buy a pair of shoes, some books, but definitely not hats.'}]


In [7]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for user_message, assistant_message in history:
        messages.append({"role": "user", "content": user_message})
        messages.append({"role": "assistant", "content": assistant_message})

    if 'belt' in message:
        messages.append({"role": "system", "content": "For added context, the store does not sell belts, \
but be sure to point out other items on sale"})

    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(model = MODEL, messages = messages, stream = True)

    response = ""

    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

In [8]:
gr.ChatInterface(fn = chat, type = "tuples").launch()

D:\Programms\Lib\site-packages\gradio\components\chatbot.py:228: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
